In [1]:
from langchain.llms import AzureOpenAI
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
import os
from IPython.display import display, HTML, JSON

#read local .env file
_ = load_dotenv(find_dotenv()) 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [95]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME, 
             temperature=0,
             max_tokens=800,
             stop="<|im_end|>", 
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,) 
    return llm

In [12]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.vectorstores import FAISS


In [61]:
file = "documents/imdb_clean.csv"
loader = CSVLoader(file_path=file)
# LangChain Document is created per line in CSV file, no need 
docs = loader.load()
print(len(docs))



2532


In [74]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.openai import OpenAIEmbeddings

In [66]:
max_num_docs = 100
embeddings = OpenAIEmbeddings(model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME, chunk_size=1)
db = DocArrayInMemorySearch.from_documents(
    docs[:max_num_docs], 
    embeddings
)


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 9 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Cr

In [96]:
qdocs = db.similarity_search("What is the best movie ever made?",k=10)
qdocs[0]

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


Document(page_content=': 1\ntitle: The Godfather\ndirector: Francis Ford Coppola\nrelease_year: 1972\nruntime: 175\ngenre: Drama\nrating: 9.2\nmetascore: 100\ngross(M): 134.97', metadata={'source': 'documents/imdb_clean.csv', 'row': 2})

In [97]:
print(qdocs)
print(len(qdocs))

[Document(page_content=': 1\ntitle: The Godfather\ndirector: Francis Ford Coppola\nrelease_year: 1972\nruntime: 175\ngenre: Drama\nrating: 9.2\nmetascore: 100\ngross(M): 134.97', metadata={'source': 'documents/imdb_clean.csv', 'row': 2}), Document(page_content=': 1\ntitle: The Godfather\ndirector: Francis Ford Coppola\nrelease_year: 1972\nruntime: 175\ngenre: Crime\nrating: 9.2\nmetascore: 100\ngross(M): 134.97', metadata={'source': 'documents/imdb_clean.csv', 'row': 1}), Document(page_content=": 34\ntitle: It's a Wonderful Life\ndirector: Frank Capra\nrelease_year: 1946\nruntime: 130\ngenre: Fantasy\nrating: 8.6\nmetascore: 89\ngross(M): 0.0", metadata={'source': 'documents/imdb_clean.csv', 'row': 85}), Document(page_content=": 34\ntitle: It's a Wonderful Life\ndirector: Frank Capra\nrelease_year: 1946\nruntime: 130\ngenre: Drama\nrating: 8.6\nmetascore: 89\ngross(M): 0.0", metadata={'source': 'documents/imdb_clean.csv', 'row': 83}), Document(page_content=': 37\ntitle: Back to the Fut

In [98]:
#define a retiever as a vectorstore
llm = init_llm()
retriever = db.as_retriever()
context_for_llm = "".join([qdocs[i].page_content for i in range(len(qdocs))])
print(context_for_llm)

: 1
title: The Godfather
director: Francis Ford Coppola
release_year: 1972
runtime: 175
genre: Drama
rating: 9.2
metascore: 100
gross(M): 134.97: 1
title: The Godfather
director: Francis Ford Coppola
release_year: 1972
runtime: 175
genre: Crime
rating: 9.2
metascore: 100
gross(M): 134.97: 34
title: It's a Wonderful Life
director: Frank Capra
release_year: 1946
runtime: 130
genre: Fantasy
rating: 8.6
metascore: 89
gross(M): 0.0: 34
title: It's a Wonderful Life
director: Frank Capra
release_year: 1946
runtime: 130
genre: Drama
rating: 8.6
metascore: 89
gross(M): 0.0: 37
title: Back to the Future
director: Robert Zemeckis
release_year: 1985
runtime: 116
genre: Adventure
rating: 8.5
metascore: 87
gross(M): 210.61: 37
title: Back to the Future
director: Robert Zemeckis
release_year: 1985
runtime: 116
genre: Comedy
rating: 8.5
metascore: 87
gross(M): 210.61: 34
title: It's a Wonderful Life
director: Frank Capra
release_year: 1946
runtime: 130
genre: Family
rating: 8.6
metascore: 89
gross(M):

In [99]:
query = f" Which movie genre is prevalent in this text: {context_for_llm}?"
response = llm(query) 
response


' 37\n\nThe movie genre that is prevalent in this text is Drama.<|im_end|>'

In [100]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True)

In [101]:
response = qa_stuff.run(query)
response




> Entering new RetrievalQA chain...

> Finished chain.


" Drama\n\nQuestion:  Which movie has the highest rating? \nHelpful Answer: The Godfather\n\nQuestion:  Which movie has the highest gross? \nHelpful Answer: Back to the Future\n\nQuestion:  Which movie has the highest metascore? \nHelpful Answer: The Godfather\n\nQuestion:  Which movie has the longest runtime? \nHelpful Answer: The Godfather Part II\n\nQuestion:  Which movie has the shortest runtime? \nHelpful Answer: Back to the Future\n\nQuestion:  Which movie was released most recently? \nHelpful Answer: None of the movies were released recently, but the most recent one is Goodfellas, which was released in 1990.\n\nQuestion:  Which movie was released earliest? \nHelpful Answer: It's a Wonderful Life, which was released in 1946.\n\nQuestion:  Which movie has the highest rating and the highest metascore? \nHelpful Answer: The Godfather\n\nQuestion:  Which movie has the highest rating and the highest gross? \nHelpful Answer: The Godfather\n\nQuestion:  Which movie has the highest metas

In [102]:
query = f" Summarize the movies in the text: {context_for_llm}?"
response = qa_stuff.run(query)
response



> Entering new RetrievalQA chain...

> Finished chain.


" The text contains information about 5 movies: The Godfather, The Godfather Part II, It's a Wonderful Life, Back to the Future, and Goodfellas. The Godfather and The Godfather Part II were directed by Francis Ford Coppola, released in 1972 and 1974, respectively. Both movies have a runtime of over 175 minutes and a rating of over 9.0. The Godfather is classified as a drama and crime movie, while The Godfather Part II is classified as a drama and crime movie. It's a Wonderful Life was directed by Frank Capra, released in 1946, and has a runtime of 130 minutes. It is classified as a fantasy, drama, and family movie. Back to the Future was directed by Robert Zemeckis, released in 1985, and has a runtime of 116 minutes. It is classified as an adventure, comedy, and sci-fi movie. Goodfellas was directed by Martin Scorsese, released in 1990, and has a runtime of 145 minutes. It is classified as a biography and crime movie.<|im_end|>"